In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
#ws = Workspace.get(name="quick-starts-ws-234286")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-235720
Azure region: eastus2
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-235720


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RW5MJVJXL to authenticate.


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = "project-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target= ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
### YOUR CODE HERE ###

In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C':choice(0.01,0.1,1,10,20,50,100,200,500), '--max_iter':choice(50,100,200)
    }
    )

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.15,delay_evaluation=5)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig("./", "train.py",  environment=sklearn_env, compute_target=compute_target)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    policy=policy,
                                    run_config=src,
                                    )

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

#hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313
Web View: https://ml.azure.com/runs/HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-235720/workspaces/quick-starts-ws-235720&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-14T08:44:09.976197][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-06-14T08:44:10.3627169Z][SCHEDULER][INFO]Scheduling job, id='HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313_0' 
[2023-06-14T08:44:10.4970142Z][SCHEDULER][INFO]Scheduling job, id='HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313_1' 
[2023-06-14T08:44:10.5970430Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313_0' 
[2023-06-14T08:44:10.6081323Z][SCHEDULER][INFO]Scheduling job, id='HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313_2' 
[2023-06-14T08:44:10.657528][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to t

{'runId': 'HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-06-14T08:44:09.44255Z',
 'endTimeUtc': '2023-06-14T08:48:40.543425Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '8f22bc90-c9c9-47f2-96be-c71ab8b87e9f',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': '27',
  'score': '0.9118361153262519',
  'best_child_run_id': 'HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClie

In [13]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

### YOUR CODE HERE ###


print("best run details :", best_run.get_details())
print('\n')
print("best run file names :", best_run.get_file_names())
print('\n')
print("best run metrics :", best_run.get_metrics())



best run details : {'runId': 'HD_fd879ccb-e4a0-4a8a-92b2-71fc4c9c0313_3', 'target': 'project-cluster', 'status': 'Completed', 'startTimeUtc': '2023-06-14T08:47:32.510805Z', 'endTimeUtc': '2023-06-14T08:47:49.906236Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '8f22bc90-c9c9-47f2-96be-c71ab8b87e9f', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '1', '--max_iter', '200'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'project-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'skl

In [14]:
model = best_run.register_model(model_name='hype_model', model_path="./")

In [15]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

tbds = TabularDatasetFactory.from_delimited_files(url)

In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    task='classification', 
    primary_metric='accuracy',
    training_data=tbds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [17]:
# Submit your automl run
automl_run = exp.submit(config=automl_config, show_output=True)
automl_run.wait_for_completion()

### YOUR CODE HERE ###

Submitting remote run.
No run_configuration provided, running on project-cluster with default configuration
Running on remote compute: project-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_7947d5e6-657e-4c3d-a179-ad39918972a2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)
print(automl_best_run_onnx.get_metrics())
OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './/best_model.onnx' )b


In [ ]:
compute_target.delete()